In [1]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.speech import SpeechConfig, SpeechRecognizer

from azure.core.credentials import AzureKeyCredential
from botframework.connector.auth import MicrosoftAppCredentials

from botbuilder.core import BotFrameworkAdapter, TurnContext, BotFrameworkAdapterSettings
from botbuilder.schema import Activity, ActivityTypes
import asyncio

In [2]:
# Initialize the text analytics config
text_analytics_key = "f66df12b36154fd68e65657ac0bd123e"
text_analytics_endpoint = "https://task8hd-language.cognitiveservices.azure.com/"
# Initialize the text recognizer
text_analytics = TextAnalyticsClient(endpoint=text_analytics_endpoint, credential = AzureKeyCredential(text_analytics_key))

# Initialize the Speech config
speech_key = "f188a4b93b75411e811116427c21b44e"
speech_region = "westeurope"
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
# Initialize the Speech recognizer
speech_recognizer = SpeechRecognizer(speech_config=speech_config)

# Initialize the Vision config
vision_key = "dfbee0faae974e81b4f9d1cf21ec9c5f"
vision_endpoint = "https://task8hd-vision.cognitiveservices.azure.com/"
# Initialize the Vision analyzer
vision_client = ComputerVisionClient(vision_endpoint, AzureKeyCredential(vision_key))

In [3]:
# Configure bot credentials
app_id = '273cc97a-38f4-44a4-a480-5f2dab7de050'
app_password = '08nLSY9ML8Q.tC0wzlAwegyJC4Gf2KzFARzpNPLNMqxsUriuh84D1Ps'

# Create Bot Framework Adapter Settings
adapter_settings = BotFrameworkAdapterSettings(app_id = app_id, app_password = app_password)

# Initialize Bot Framework Adapter with the settings
adapter = BotFrameworkAdapter(adapter_settings)

In [6]:
async def process_message(activity: Activity):
    text = activity.text
    if activity.attachments:
        attachment = activity.attachments[0]
        if attachment.content_type.startswith("image/"):
            # Image input, use Computer Vision
            image_url = attachment.content_url
            result = vision_client.describe_image(image_url)
            response = f"Computer Vision detected: {result.captions[0].text}"
        else:
            response = "Unsupported attachment type"
    else:
        # Text input, use Text Analytics
        sentiment = text_analytics.sentiment(text)
        response = f"Sentiment: {sentiment.score}"
    
    return response

In [7]:
async def turn_handler(turn_context: TurnContext):
    if turn_context.activity.type == ActivityTypes.message:
        response = await process_message(turn_context.activity)
        await turn_context.send_activity(response)

In [8]:
async def on_turn(turn_context: TurnContext):
    await turn_handler(turn_context)

In [9]:
# Start the bot
async def main():
    async def process_activity(activity: Activity):
        await adapter.process_activity(activity, on_turn)
    
    await process_activity(Activity(type="invoke", name="webhook", value=None))

In [ ]:
if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

In [ ]:
async def on_message_activity(self, turn_context: TurnContext):
    if turn_context.activity.text:
        # Handle text input
        response = handle_text_input(turn_context.activity.text)
    elif turn_context.activity.attachments:
        for attachment in turn_context.activity.attachments:
            if attachment.content_type.startswith('audio'):
                # Use Azure Speech service to transcribe speech input
                result = speech_recognizer.recognize_once(stream=attachment.content)
                if result.reason == speechsdk.ResultReason.RecognizedSpeech:
                    transcribed_text = result.text
                    # Handle transcribed text
                    response = handle_text_input(transcribed_text)
                else:
                    response = "Speech recognition failed"
            elif attachment.content_type.startswith('image'):
                # Use Azure Computer Vision service to analyze image
                image_stream = BytesIO(await attachment.open_stream())
                response = analyze_image(image_stream)
            else:
                response = "Unsupported attachment type"
    else:
        response = "Unsupported input type"

    await turn_context.send_activity(response)


In [ ]:
def analyze_image(image_stream):
    # Analyze image using Computer Vision service
    image_features = ["categories", "tags", "description"]
    analysis = vision_client.analyze_image_in_stream(image_stream, visual_features=image_features)

    # Process analysis results
    description = analysis.description.captions[0].text if analysis.description.captions else "No description found"
    tags = ', '.join([tag.name for tag in analysis.tags]) if analysis.tags else "No tags found"

    return f"Description: {description}\nTags: {tags}"